In [ ]:
# 라이브러리 불러오기 및 함수화
import re
import numpy as np
import pandas as pd

from konlpy.tag import Komoran
from pykospacing import Spacing
from gensim.models import Word2Vec

komoran = Komoran()
spacing = Spacing()

In [ ]:
def read_data(filename):
    with open(filename, 'r', encoding="cp949") as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

analysis_data = pd.read_csv(r'..\..\..\data\combined_questions.csv', encoding='cp949')

# 불용어 리스트 불러오기
korean_stopwords_path = '../../../data/stopwords-ko.txt'
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
# 문장 전처리
def preprocessing(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = spacing(text)
    return text

preprocessing_data = []
for data in analysis_data:
    data = ''.join(data)
    data = preprocessing(data)
    preprocessing_data.append(data)
    
# 형태소 분리
komoran_data = []
for data in preprocessing_data:
    morphs = komoran.morphs(data)
    komoran_data.append(morphs)

# 불용어를 제거한 토큰 리스트 생성
filtered_tokens = []
for data in komoran_data:
    for token in data:
        if token not in stopwords:
            filtered_tokens.append(token)            
token_df = pd.DataFrame(filtered_tokens)

In [ ]:
# Word2Vec
model = Word2Vec(sentences=token_df[0], vector_size=100, window=4, hs=1, min_count=2, sg=1)

def get_document_vector(tokens, model):
    word_vectors = [model.wv[token] for token in tokens if token in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# 문서 벡터화
analysis_data_df = pd.DataFrame(analysis_data, columns=["text"])
analysis_data_df['vector'] = analysis_data_df['text'].apply(lambda x: get_document_vector(x, model))